# Regression Model B: Field 4

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 4 (F4)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E4" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E4" ),]
dE4=d4[which(d4$Estate=="E4" ),]
dE5=d5[which(d5$Estate=="E4" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
17,1,1,1,1,1,1,1,1,1,1,1,0
12,1,1,1,1,1,1,1,1,1,0,0,2
4,1,1,1,1,1,1,1,0,0,1,1,2
11,1,1,1,1,1,1,1,0,0,0,0,4
,0,0,0,0,0,0,0,15,15,23,23,76


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost
  3   3  prun.time  prun.cost  manuring.time  manuring.cost
  3   4  prun.time  prun.cost  manuring.time  manuring.cost
  3   5  prun.time  prun.cost  manuring.time  manuring.cost
  4   1  prun.time  prun.cost  manuring.time  manuring.cost
  4   2  prun.time  

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            23            23            15            15 
      pd.time       pd.cost 
            0             0 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost 
            2             3             4             5 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1             1             1       1
prun.cost          1         1  

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper",col=brewer.pal(n=8, name="RdYlBu"))


## Regression

Fit the data using multiple regression for all variables. 

In [10]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=df2)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-225.13  -46.29  -15.16   66.83  185.82 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
(Intercept)   472.445302 153.065207   3.087  0.00394 **
rainfall        0.007268   0.039976   0.182  0.85679   
foliar        -10.615707   4.908759  -2.163  0.03749 * 
prun.time     -35.728653  34.092635  -1.048  0.30183   
prun.cost       0.004554   0.006939   0.656  0.51592   
manuring.time -28.201243  55.930139  -0.504  0.61726   
manuring.cost  -0.001490   0.001298  -1.148  0.25882   
pd.time        24.346665  22.162770   1.099  0.27947   
pd.cost         0.002926   0.009546   0.307  0.76101   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 92.93 on 35 degrees of freedom
Multiple R-squared:  0.2579,	Adjusted R-squared:  0.08826 
F-statis

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [11]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,193.11285,193.11285,0.022363732,0.88198196
foliar,1,49570.28908,49570.28908,5.740563846,0.02205756
prun.time,1,11984.67346,11984.67346,1.387903610,0.24670771
prun.cost,1,1420.62342,1420.62342,0.164517488,0.68749993
manuring.time,1,24.09802,24.09802,0.002790709,0.95816999
manuring.cost,1,26312.11039,26312.11039,3.047114560,0.08965509
pd.time,1,14706.91076,14706.91076,1.703156503,0.20039090
pd.cost,1,811.44494,811.44494,0.093970633,0.76100503
Residuals,35,302228.17209,8635.09063,NA,NA


### Using stepwise

In [12]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=406.73
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- rainfall       1       285 302514 404.77
- pd.cost        1       811 303040 404.85
- manuring.time  1      2195 304424 405.05
- prun.cost      1      3719 305948 405.27
- prun.time      1      9484 311712 406.09
- pd.time        1     10421 312649 406.22
- manuring.cost  1     11377 313605 406.35
<none>                       302228 406.73
- foliar         1     40385 342613 410.25

Step:  AIC=404.77
yield ~ foliar + prun.time + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- pd.cost        1       840 303354 402.89
- manuring.time  1      2245 304759 403.10
- prun.cost      1      3873 306387 403.33
- prun.time      1      9551 312065 404.14
- pd.time        1     10259 312773 404.24
- manuring.cost  1     11890 314403 404.47
<none>                    


Call:
lm(formula = yield ~ foliar + manuring.cost, data = df2)

Residuals:
     Min       1Q   Median       3Q      Max 
-215.411  -55.864   -6.834   49.017  204.237 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   426.249845  30.690917  13.888   <2e-16 ***
foliar         -9.763450   3.993126  -2.445   0.0189 *  
manuring.cost  -0.002156   0.001059  -2.037   0.0482 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 89 on 41 degrees of freedom
Multiple R-squared:  0.2025,	Adjusted R-squared:  0.1636 
F-statistic: 5.206 on 2 and 41 DF,  p-value: 0.009669


       foliar manuring.cost 
     1.000777      1.000777 



Call:
lm(formula = yield ~ foliar + manuring.cost, data = df2)

Residuals:
     Min       1Q   Median       3Q      Max 
-215.411  -55.864   -6.834   49.017  204.237 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   426.249845  30.690917  13.888   <2e-16 ***
foliar         -9.763450   3.993126  -2.445   0.0189 *  
manuring.cost  -0.002156   0.001059  -2.037   0.0482 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 89 on 41 degrees of freedom
Multiple R-squared:  0.2025,	Adjusted R-squared:  0.1636 
F-statistic: 5.206 on 2 and 41 DF,  p-value: 0.009669


## Prediction

Predict with the resample values.

In [13]:
set.seed(123)
df3=sample(1:nrow(df2),12)
df4=df2[df3,]
estimate=predict(selectedMod,newdata=df4)
estimate
sum.estimate=sum(estimate)
sum.estimate

df5=data.frame(estimate,sum.estimate)

13       34       18       37       38        2       21       43 
373.7093 246.6680 314.9450 368.2409 390.6491 405.6420 295.6017 336.2772 
      20       16       33       15 
350.5316 378.2146 290.1333 344.2353

[1] 4094.848